In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import classification_report,confusion_matrix
import random
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

In [2]:
#%%
from mysql import connector

#%%
cnx = connector.connect(
    user='root', 
    password='paramore',
    host='127.0.0.1',
    database='paspe'
)

#%%
def query(sql):
    cursor = cnx.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()
    cursor.close()
    return data

In [3]:
#%%
tmp = query("""
    SELECT b.IdValorParametro campo_id
      FROM paspe.paspe_campos a
INNER JOIN paspe.extidi_valorparametro b ON a.IdParametro = b.IdParametro
     WHERE a.UtilizaParaCalculo = 1
""")
campos = []
strcampos = ""
for row in tmp:
    campos.append(row[0])
    strcampos = strcampos + "`" + str(row[0]) + "`, "
strcampos = strcampos + "`conteo`"

In [104]:
steps = 20
sql = """
      SELECT * FROM (
      SELECT {0}
        FROM encuestas
       WHERE Id_alumn_programa > 0
         AND conteo > 30 and deserto=0
       LIMIT 1000) a
       UNION ALL
       SELECT * FROM (
       SELECT {0}
        FROM encuestas
       WHERE Id_alumn_programa > 0
         AND conteo > 30 and deserto=1
       LIMIT 1000) a
""".format(strcampos)
x = np.array(query(sql))
sql = """
    SELECT * FROM (
    SELECT deserto
      FROM encuestas
     WHERE Id_alumn_programa > 0 and deserto=0
       AND conteo > 30
     LIMIT 1000) a
     UNION ALL
    SELECT * FROM (
    SELECT deserto
      FROM encuestas
     WHERE Id_alumn_programa > 0 and deserto=1
       AND conteo > 30
     LIMIT 1000) a
"""
y = np.array(query(sql))

In [105]:
print(x.shape)
print(y.shape)

(2000, 166)
(2000, 1)


In [106]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [107]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1600, 166)
(1600, 1)
(400, 166)
(400, 1)


In [108]:
#Defining the model 

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adam


model = Sequential()

#Sigmoid, tanh, Softmax, ReLU, Leaky ReLU

model = Sequential()

#sigmoid, tanh, softmax, ReLU, Leaky ReLU

model.add(Dense(1024,input_shape=(166,),activation='sigmoid'))
model.add(Dense(512,activation='sigmoid'))
model.add(Dense(256,activation='sigmoid'))
model.add(Dense(128,activation='sigmoid'))
model.add(Dense(64,activation='sigmoid'))
model.add(Dense(2,activation='softmax'))

model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 1024)              171008    
_________________________________________________________________
dense_22 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_23 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_24 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_25 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 130       
Total params: 868,418
Trainable params: 868,418
Non-trainable params: 0
_________________________________________________________________


In [109]:
from keras.utils import to_categorical

In [112]:
def aprender(x_train, y_train):
    model.fit(x_train, to_categorical(y_train),epochs=10)

In [113]:
aprender(x_train, y_train[:,0])

Epoch 1/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.7005 - acc: 0.4869
Epoch 2/10
1600/1600 [==============================] - 4s 3ms/step - loss: 0.6979 - acc: 0.4919
Epoch 3/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.7020 - acc: 0.4894
Epoch 4/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.7008 - acc: 0.4856
Epoch 5/10
1600/1600 [==============================] - 4s 3ms/step - loss: 0.6982 - acc: 0.4794
Epoch 6/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.7018 - acc: 0.5131
Epoch 7/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.7072 - acc: 0.5019
Epoch 8/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.7005 - acc: 0.4969
Epoch 9/10
1600/1600 [==============================] - 6s 3ms/step - loss: 0.6980 - acc: 0.4831
Epoch 10/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.6959 - acc: 0.4994


In [24]:
to_categorical(y_train)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [25]:
y_train[:,0]

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
x_train

array([[ 0,  1,  0, ...,  0,  1, 45],
       [ 0,  1,  0, ...,  0,  1, 47],
       [ 1,  0,  0, ...,  0,  0, 43],
       ...,
       [ 1,  0,  0, ...,  0,  0, 41],
       [ 0,  1,  0, ...,  1,  0, 44],
       [ 1,  0,  0, ...,  0,  1, 48]])

In [67]:
y_pred = model.predict(x_test)
y_pred

array([[1.8201121e-05, 9.9998176e-01],
       [1.8201121e-05, 9.9998176e-01],
       [1.8201121e-05, 9.9998176e-01],
       ...,
       [1.8201121e-05, 9.9998176e-01],
       [1.8201121e-05, 9.9998176e-01],
       [1.8201121e-05, 9.9998176e-01]], dtype=float32)

In [47]:
y_pred = model.predict(x_train)
#y_pred[0] = [0,1]
y_test_class = x_train[:,0] #np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

print(y_pred_class)
print(y_test_class)

#print(y_pred_class)

#print(np.sum(y_test))
#print(np.sum(y_pred))

#print(np.sum(y_test_class))
#print(np.sum(y_pred_class))

print(classification_report(y_test_class,y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))

[0 0 0 ... 0 0 0]
[1 1 0 ... 0 1 1]
             precision    recall  f1-score   support

          0       0.42      1.00      0.59      6684
          1       0.00      0.00      0.00      9316

avg / total       0.17      0.42      0.25     16000

[[6684    0]
 [9316    0]]


/home/juliotorres/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [82]:
n = 0 #random.randint(0, 1)
x = np.array(query("""
    SELECT {0}
        FROM encuestas
        WHERE Id_alumn_programa > 0
          AND conteo > 30 
          AND deserto = {1} 
""".format(strcampos, str(n))))
y = np.array(query("""
    SELECT deserto
      FROM encuestas
     WHERE Id_alumn_programa > 0
       AND conteo > 30 
       AND deserto = {0}
""".format(str(n))))
y_pred = model.predict(x)
y_test_class = np.argmax(y,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)


In [85]:
print(n)
cnt = 0
#for i in range(0, len(y_pred)):
#    if y[i] == y_pred[i]:
#        cnt = cnt + 1
#print(cnt)
print(y)
print(y_pred)
#print(np.argmax(y_pred,axis=0))
#print(np.argmax(y,axis=0))

0
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
[[1.8201121e-05 9.9998176e-01]
 [1.8201121e-05 9.9998176e-01]
 [1.8201121e-05 9.9998176e-01]
 ...
 [1.8201121e-05 9.9998176e-01]
 [1.8201121e-05 9.9998176e-01]
 [1.8201121e-05 9.9998176e-01]]


In [84]:
y_test_class = y[:,0] #np.argmax(y,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)


print(classification_report(y_test_class,y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00     25007
          1       0.00      0.00      0.00         0

avg / total       0.00      0.00      0.00     25007

[[    0 25007]
 [    0     0]]


/home/juliotorres/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/juliotorres/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [56]:
np.sum(y_pred)

4092.9995

In [57]:
y_pred_class[0]

0

In [58]:
np.sum(y_test_class)

0

In [59]:
np.sum(y_pred_class)

0